For an existing database, gets the counts of departures per agency.

In [2]:
# load libraries
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine, text
import zipfile

In [3]:
# Welcher Zip?
source = "delfi" # where is the feed file? delfi or gtfs.de?
zipname = '20220425_fahrplaene_gesamtdeutschland_gtfs' # name of GTFS zipfile
# define paths

workingdir = "../../data/interim/" 

In [4]:
# choose file-based output connection
dbpath = '{0}{1}.db'.format(workingdir,zipname)
# set up DB connection
db = create_engine('sqlite:///' + dbpath)

In [19]:
# Get stuff from zip
zippath = "../../data/raw/delfi/" + zipname + ".zip"

# set up zip file as default for functions
zf = zipfile.ZipFile(zippath) # this is the raw stuff

# load things
calendar_df = pd.read_csv(zf.open("calendar.txt"))
calendar_dates_df = pd.read_csv(zf.open("calendar_dates.txt"))
agency_df = pd.read_csv(zf.open("agency.txt"))

In [26]:
%%time
print(dt.datetime.now())
agency_count_df = pd.read_sql_query(
    'SELECT routes.agency_id, SUM(n) AS n '
    # 'SELECT agency.agency_id, agency.agency_name, SUM(n) AS n '
    'FROM routes '
    'LEFT JOIN ( '
        'SELECT trips.trip_id, trips.route_id, SUM(days_count) AS n '
        'FROM trips '
        'LEFT JOIN stop_times ON trips.trip_id = stop_times.trip_id '
        'LEFT JOIN calendar ON trips.service_id = calendar.service_id '
        'GROUP BY trips.trip_id '
    ') as trip_counts '
    'ON trip_counts.route_id = routes.route_id '
    'GROUP BY routes.agency_id',
#    'RIGHT JOIN agency ON routes.agency_id = agency.agency_id '
#    'GROUP BY agency.agency_id ',
    db
)
   

2022-07-26 11:55:08.268311
CPU times: user 1min 11s, sys: 8.05 s, total: 1min 19s
Wall time: 1min 24s


In [27]:
agency_count_df

,agency_id,n
0,7489,235113.0
1,7499,7156221.0
2,7500,6394061.0
3,7501,13330.0
4,7502,2601379.0
...,...,...
1067,14064,24550.0
1068,14082,335529.0
1069,14083,132367.0
1070,14084,474043.0


In [36]:
# join agency
df = agency_df.merge(agency_count_df, on='agency_id', how='left')

finally, days

In [34]:
# Feed days function
def feedDays(calendar_df, calendar_dates_df):
    ''' Enriches counted dataframe with average daily count for each stop,
    using the feed's calendar information to infer the number of days
    '''
    # calculate
    startdate =  min(pd.to_datetime(calendar_df.start_date,format="%Y%m%d"))
    enddate = max(pd.to_datetime(calendar_df.end_date,format="%Y%m%d"))
    excdates = pd.to_datetime(calendar_dates_df.date,format="%Y%m%d")

    firstdate = min(startdate, min(excdates))
    lastdate = max(enddate, max(excdates))

    ndays = (lastdate - firstdate).days

    print('First date:\t',firstdate)
    print('Last date:\t',lastdate)
    print('Total days:\t',ndays)
    return(ndays)

In [35]:
ndays = feedDays(calendar_df, calendar_dates_df)

First date:	 2022-04-11 00:00:00
Last date:	 2022-12-10 00:00:00
Total days:	 243


In [39]:
df['n_day'] = df.loc[:,'n']/ndays

In [47]:
df.loc[df.agency_name.str.contains("Fernverkehr|Flix|Eurocity|SBB|Hamburg-Köln-Express|SNCF|Trenitalia|Österreichische") & 
      ~df.agency_name.str.contains("Codesharing|SBB GmbH"),['agency_id','agency_name','n','n_day']].sort_values('agency_name'
)#.to_csv('../../data/processed/'+zipname+'_n-agency-fv.csv')

,agency_id,agency_name,n,n_day
923,12681,DB Fernverkehr AG,2244147.0,9235.172840
669,10393,DB Fernverkehr Bus,NaN,NaN
687,10451,DB/SBB/TI Eurocity Milano,8282.0,34.082305
1120,13794,FlixBus-de,860203.0,3539.930041
920,12636,FlixTrain,NaN,NaN
1121,13796,FlixTrain-de,25418.0,104.600823
716,10874,Hamburg-Köln-Express,NaN,NaN
690,10785,SBB,58838.0,242.131687
916,12623,SNCF,4845.0,19.938272
917,12624,Trenitalia,NaN,NaN


In [46]:
df.loc[:,['agency_id','agency_name','n','n_day']].sort_values('agency_name'
).to_csv('../../data/processed/'+zipname+'_n-agency-alle.csv')